<a href="https://colab.research.google.com/github/pawanaichra/ai61002s19/blob/master/localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import tempfile
import pandas as pd
from skimage import io,transform
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
path = "/content/drive/My Drive/"
torch.set_default_tensor_type('torch.DoubleTensor')
class ToTensor(object):
    def __call__(self, sample):
        image, box = sample['image'], sample['box']
        image = np.dot(image[...,:3], [0.299, 0.587, 0.114])
        image = (image-image.min())/(image.max()-image.min())
        # torch image: C X H X W
        image=image.reshape(1,640, 480)
        return {'image': torch.from_numpy(image),
                'box': torch.from_numpy(box)}

class dataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.df.iloc[idx, 0])
        image = io.imread(img_name)
        box = np.array([self.df.iloc[idx, 1], self.df.iloc[idx, 3], self.df.iloc[idx, 2], self.df.iloc[idx, 4]])
        sample = {'image': image, 'box': box}
        if self.transform:
            sample = self.transform(sample)

        return sample
data = dataset(csv_file = os.path.join(path, "flip/training.csv"), root_dir = os.path.join(path, "images/"), transform=transforms.Compose([ToTensor()]))

#imge size 640*480
class convnet(nn.Module):
    def __init__(self):
        super(convnet, self).__init__()
        self.conv1 = nn.Conv2d(1, 4, kernel_size=5)
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=3)
        self.conv2 = nn.Conv2d(4, 8, kernel_size=5)
        self.pool2 = nn.MaxPool2d(kernel_size=3,stride=3)   
        self.conv3 = nn.Conv2d(8, 16, kernel_size=5)
        self.pool3 = nn.MaxPool2d(kernel_size=3,stride=3) 
        self.fc1 = nn.Linear(5040, 1000)
        self.fc2 = nn.Linear(1000, 84)
        self.fc3 = nn.Linear(84, 4)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)      
        x = x.view(-1, 5040)
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        m = torch.tensor([[640, 480, 640, 480]]).type(torch.DoubleTensor).cuda()
        a = x*m
        c = torch.zeros(a.shape[0], 2)
        d = torch.zeros(a.shape[0], 2)
        for idx, x in enumerate(a):
            d1 = x[0] ** 2 + x[1] ** 2
            d2 = x[2] ** 2 + x[3] ** 2
            d3 = x[0] ** 2 + x[3] ** 2
            d4 = x[2] ** 2 + x[1] ** 2
            dmin = min(d1, d2, d3, d4)
            dmax = max(d1, d2, d3, d4)
            if d1 == dmin:
                c[idx] = torch.tensor([x[0], x[1]])
            elif d2 == dmin:
                c[idx] = torch.tensor([x[2], x[3]])
            elif d3 == dmin:
                c[idx] = torch.tensor([x[0], x[3]])
            elif d4 == dmin:
                c[idx] = torch.tensor([x[2], x[1]])
            if d1 == dmax:
                d[idx] = torch.tensor([x[0], x[1]])
            elif d2 == dmax:
                d[idx] = torch.tensor([x[2], x[3]])
            elif d3 == dmax:
                d[idx] = torch.tensor([x[0], x[3]])
            elif d4 == dmax:
                d[idx] = torch.tensor([x[2], x[1]])
        x = torch.cat((c,d),1)
        return x
    def sigmoid(self, z):
        return 1/(1+torch.exp(-z))
      
      
      

net = convnet()
use_gpu = torch.cuda.is_available()
if use_gpu: 
    net = net.cuda()
    
    
optimizer = optim.Adadelta(net.parameters(), lr=0.01)
def iou(box_a, box_b):
  A = box_a.size(0)
  B = box_b.size(0)
  max_xy = torch.min(box_a[:, 2:].unsqueeze(1).expand(A, B, 2),
                       box_b[:, 2:].unsqueeze(0).expand(A, B, 2))
  min_xy = torch.max(box_a[:, :2].unsqueeze(1).expand(A, B, 2),
                       box_b[:, :2].unsqueeze(0).expand(A, B, 2))
  inter = torch.clamp((max_xy - min_xy), min=0)
  inter =inter[:, :, 0] * inter[:, :, 1]
  area_a = ((box_a[:, 2]-box_a[:, 0]) *
              (box_a[:, 3]-box_a[:, 1])).unsqueeze(1).expand_as(inter)
  area_b = ((box_b[:, 2]-box_b[:, 0]) *
              (box_b[:, 3]-box_b[:, 1])).unsqueeze(0).expand_as(inter)
  union = area_a + area_b - inter
  return ((inter / union)*100/float(A*A)).sum()

def criterion(output, labels):
  return (100-iou(output, labels))

# criterion = nn.MSELoss()

In [0]:
dataloader = DataLoader(data, batch_size=4, shuffle=True, num_workers=4)

In [0]:
# net=convnet().cuda()
# net.load_state_dict(torch.load(path+"flip/weight/tmpq7_m34ps"))
# net.eval()

In [0]:

iterations =5
trainLoss = []
testAcc = []

# import pylab as pl
# from IPython import display
# fig=plt.figure()
# ax=fig.add_subplot(111)
# ax.set_xlim(0,iterations)


for epoch in range(iterations):
    runningLoss = 0 
    count=0
    net.train(True)
    for data in dataloader:
      inputs = data["image"]
      labels = data["box"]
      if use_gpu:
          inputs = Variable(inputs.cuda())
          labels = Variable(labels.type(torch.DoubleTensor).cuda())
      else:
          inputs = Variable(inputs)
          labels = Variable(labels) 
      optimizer.zero_grad()   
      outputs = net(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      runningLoss += loss.data.item()
      count+=1
      print(runningLoss/count)
#     with tempfile.NamedTemporaryFile(dir = path+"flip/weight/", delete = False) as f:
#       torch.save(net.state_dict(), f)    
    avgTrainLoss = runningLoss/count
    trainLoss.append(avgTrainLoss)
#     ax.plot(range(len(trainLoss)),trainLoss,'r-',label='Training')
#     display.clear_output(wait=True)
#     display.display(pl.gcf())
    
    
#     with torch.no_grad():
#         running_correct = 0
#         count1=0
#         for data in test_set:
#             inputs, labels = data
#             if use_gpu:
#                 inputs = Variable(inputs.cuda()) 
#                 labels = labels.type(torch.DoubleTensor).cuda()
#             else:
#                 inputs = Variable(inputs)
#             outputs = net(inputs)
#             running_correct += iou(outputs,labels).item()
#             count1+=1
#         avgTestAcc = float(running_correct)/count1
#         testAcc.append(avgTestAcc)
#         ax.plot(range(len(testAcc)),testAcc,'g-',label='Training')
#         display.clear_output(wait=True)
#         display.display(pl.gcf())

In [0]:
#saving prediction
import csv
predict_set = pd.read_csv(os.path.join(path , "flip/test.csv"))
f = open(path+"flip/out1.csv", "w")
writer = csv.writer(f)
row = []
row.append(["image_name","x1",	"x2",	"y1",	"y2"])
with torch.no_grad():
  for index, data in predict_set.iterrows():
    try:
      image = io.imread(os.path.join(path, "images/"+data[0]))
      image = np.dot(image[...,:3], [0.299, 0.587, 0.114])
      image = (image-image.min())/(image.max()-image.min())
      image=image.reshape(1,1,640, 480)
      image= torch.from_numpy(image)
      inputs = Variable(image.type(torch.DoubleTensor).cuda())
      outputs = net(inputs)
      row.append([data["image_name"], outputs[0][0].item(), outputs[0][2].item(), outputs[0][1].item(), outputs[0][3].item()])
    except:
      row.append([data["image_name"], 192, 448, 144, 336])   
writer.writerows(row)
f.close()

    